<a href="https://colab.research.google.com/github/briyanyehezkhiel/Klasifikasi_Submission_Akhir_BMLP_Briyan_Yehezkhiel/blob/main/%5BKlasifikasi%5D_Submission_Akhir_BMLP_Briyan_Yehezkhiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Penting**
- Jangan mengubah atau menambahkan cell text yang sudah disediakan, Anda hanya perlu mengerjakan cell code yang sudah disediakan.
- Pastikan seluruh kriteria memiliki output yang sesuai, karena jika tidak ada output dianggap tidak selesai.
- Misal, Anda menggunakan df = df.dropna() silakan gunakan df.isnull().sum() sebagai tanda sudah berhasil. Silakan sesuaikan seluruh output dengan perintah yang sudah disediakan.
- Pastikan Anda melakukan Run All sebelum mengirimkan submission untuk memastikan seluruh cell berjalan dengan baik.
- Pastikan Anda menggunakan variabel df dari awal sampai akhir dan tidak diperbolehkan mengganti nama variabel tersebut.
- Hapus simbol pagar (#) pada kode yang bertipe komentar jika Anda menerapkan kriteria tambahan
- Biarkan simbol pagar (#) jika Anda tidak menerapkan kriteria tambahan
- Pastikan Anda mengerjakan sesuai section yang sudah diberikan tanpa mengubah judul atau header yang disediakan.

# **1. Import Library**
Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
#Type your code here
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**
Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [ ]:
# Gunakan dataset hasil clustering yang memiliki fitur Target
# Silakan gunakan dataset data_clustering jika tidak menerapkan Interpretasi Hasil Clustering [Advanced]
# Silakan gunakan dataset data_clustering_inverse jika menerapkan Interpretasi Hasil Clustering [Advanced]
# Lengkapi kode berikut
df = pd.read_csv("data_clustering_inverse.csv")

In [ ]:
# Tampilkan 5 baris pertama dengan function head.
display(df.head())

,TransactionAmount,TransactionDate,TransactionType,Location,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,TransactionAmount_Bin,CustomerAge_Bin,TransactionAmount_Bin_Encoded,CustomerAge_Bin_Encoded,Target
0,14.09,2023-04-11 16:29:14,Debit,San Diego,ATM,70.0,Doctor,81.0,1.0,5112.21,2024-11-04 08:08:08,0,4,0,4,1
1,376.24,2023-06-27 16:44:19,Debit,Houston,ATM,68.0,Doctor,141.0,1.0,13758.91,2024-11-04 08:09:35,0,4,0,4,2
2,126.29,2023-07-10 18:16:08,Debit,Mesa,Online,19.0,Student,56.0,1.0,1122.35,2024-11-04 08:07:04,0,0,0,0,2
3,184.50,2023-05-05 16:32:11,Debit,Raleigh,Online,26.0,Student,25.0,1.0,8569.06,2024-11-04 08:09:06,0,0,0,0,1
4,13.45,2023-10-16 17:51:24,Credit,Atlanta,Online,45.0,Student,198.0,1.0,7429.40,2024-11-04 08:06:39,0,2,0,2,0


# **3. Data Splitting**
Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [ ]:
# Menggunakan train_test_split() untuk melakukan pembagian dataset.

X = df.drop('Target', axis=1)
y = df['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data splitting complete.")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

Data splitting complete.
X_train shape: (2011, 15)
X_test shape: (503, 15)
y_train shape: (2011,)
y_test shape: (503,)


# **4. Membangun Model Klasifikasi**
Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Menggunakan algoritma klasifikasi yaitu Decision Tree.
2. Latih model menggunakan data yang sudah dipisah.

In [ ]:
# Buatlah model klasifikasi menggunakan Decision Tree
model = DecisionTreeClassifier(random_state=42)

# Preprocessing steps (converting date columns and one-hot encoding)
X_train_processed = X_train.copy()
X_test_processed = X_test.copy()

for col in ['TransactionDate', 'PreviousTransactionDate']:
    X_train_processed[col] = pd.to_datetime(X_train_processed[col], errors='coerce').astype('int64') // 10**9
    X_test_processed[col] = pd.to_datetime(X_test_processed[col], errors='coerce').astype('int64') // 10**9

categorical_cols = X_train_processed.select_dtypes(include=['object']).columns

X_train_processed = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_processed = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

train_cols = X_train_processed.columns
test_cols = X_test_processed.columns
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_processed[c] = 0
missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_train_processed[c] = 0
X_test_processed = X_test_processed[train_cols]

model.fit(X_train_processed, y_train)

DecisionTreeClassifier(random_state=42)

In [ ]:
# Menyimpan Model
joblib.dump(model, 'decision_tree_model.h5')

['decision_tree_model.h5']

# **5. Memenuhi Kriteria Skilled dan Advanced dalam Membangun Model Klasifikasi**



**Biarkan kosong jika tidak menerapkan kriteria skilled atau advanced**

In [ ]:
# Melatih model menggunakan algoritma klasifikasi selain Decision Tree.
rf_model = RandomForestClassifier(random_state=42)

X_train_processed = X_train.copy()
X_test_processed = X_test.copy()

for col in ['TransactionDate', 'PreviousTransactionDate']:
    X_train_processed[col] = pd.to_datetime(X_train_processed[col]).astype('int64') // 10**9
    X_test_processed[col] = pd.to_datetime(X_test_processed[col]).astype('int64') // 10**9

categorical_cols = X_train_processed.select_dtypes(include=['object']).columns

X_train_processed = pd.get_dummies(X_train_processed, columns=categorical_cols, drop_first=True)
X_test_processed = pd.get_dummies(X_test_processed, columns=categorical_cols, drop_first=True)

train_cols = X_train_processed.columns
test_cols = X_test_processed.columns
missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test_processed[c] = 0
missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_train_processed[c] = 0
X_test_processed = X_test_processed[train_cols]

rf_model.fit(X_train_processed, y_train)

print("Random Forest model trained successfully.")

Random Forest model trained successfully.


In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.

y_pred_dt = model.predict(X_test_processed)

y_pred_rf = rf_model.predict(X_test_processed)

print("Decision Tree Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Precision:", precision_score(y_test, y_pred_dt, average='weighted', zero_division=0))
print("Recall:", recall_score(y_test, y_pred_dt, average='weighted', zero_division=0))
print("F1-Score:", f1_score(y_test, y_pred_dt, average='weighted', zero_division=0))
print("-" * 30)

print("Random Forest Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Precision:", precision_score(y_test, y_pred_rf, average='weighted', zero_division=0))
print("Recall:", recall_score(y_test, y_pred_rf, average='weighted', zero_division=0))
print("F1-Score:", f1_score(y_test, y_pred_rf, average='weighted', zero_division=0))

Decision Tree Model Evaluation:
Accuracy: 0.9980119284294234
Precision: 0.9980245111608828
Recall: 0.9980119284294234
F1-Score: 0.9980122839977794
------------------------------
Random Forest Model Evaluation:
Accuracy: 0.9940357852882704
Precision: 0.9940698193943429
Recall: 0.9940357852882704
F1-Score: 0.9940352214250312


In [ ]:
# Menyimpan Model Selain Decision Tree
# Model ini bisa lebih dari satu
joblib.dump(rf_model, 'explore_random_forest_classification.h5')

['explore_random_forest_classification.h5']

Hyperparameter Tuning Model

Pilih salah satu algoritma yang ingin Anda tuning

In [ ]:
# Lakukan Hyperparameter Tuning dan Latih ulang.
# Lakukan dalam satu cell ini saja.

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=3,
                           scoring='accuracy',
                           n_jobs=-1)

grid_search.fit(X_train_processed, y_train)

best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

print("Hyperparameter tuning complete.")
print("Best parameters found:", best_params)

tuned_model = best_rf_model

print("Model retrained with best parameters.")

Hyperparameter tuning complete.
Best parameters found: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Model retrained with best parameters.


In [ ]:
# Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.

y_pred_tuned = tuned_model.predict(X_test_processed)

print("Tuned Random Forest Model Evaluation:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned))
print("Precision:", precision_score(y_test, y_pred_tuned, average='weighted', zero_division=0))
print("Recall:", recall_score(y_test, y_pred_tuned, average='weighted', zero_division=0))
print("F1-Score:", f1_score(y_test, y_pred_tuned, average='weighted', zero_division=0))

Tuned Random Forest Model Evaluation:
Accuracy: 0.9940357852882704
Precision: 0.9940698193943429
Recall: 0.9940357852882704
F1-Score: 0.9940352214250312


In [ ]:
# Menyimpan Model hasil tuning

joblib.dump(tuned_model, 'tuning_classification.h5')

['tuning_classification.h5']

End of Code